In [43]:
import requests
import pandas as pd

API_KEY = "eefb8c41863048df9e948694f8b52e6a"
url = f"https://newsapi.org/v2/top-headlines?language=en&apiKey={API_KEY}"

response = requests.get(url).json()
headlines = [article['title'] for article in response['articles']]
df = pd.DataFrame(headlines, columns=['headline'])


In [45]:
# Example labeled data
# 1 = Positive, 0 = Negative
data = spark.createDataFrame([
    ("Stock prices soar today", 1),
    ("Market crashes amid fears", 0),
    ("Tech sector booming", 1),
    ("Economic slowdown expected", 0)
], ["headline", "label"])


In [50]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


In [51]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Define UDF to convert sentiment to 0/1
def sentiment_label(text):
    score = analyzer.polarity_scores(text)['compound']
    return 1 if score >= 0 else 0

sentiment_udf = udf(sentiment_label, IntegerType())
data = data.withColumn("label", sentiment_udf(data["headline"]))


In [52]:
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, lr])
model = pipeline.fit(data)  # should work now
predictions = model.transform(data)
predictions.show()


+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|            headline|label|               words|            filtered|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Rihanna welcomes ...|    1|[rihanna, welcome...|[rihanna, welcome...|(262144,[4308,748...|(262144,[4308,748...|[-20.513237959292...|[1.23371245096356...|       1.0|
|Starbucks announc...|    1|[starbucks, annou...|[starbucks, annou...|(262144,[17128,38...|(262144,[17128,38...|[-19.591278896050...|[3.10181349177638...|       1.0|
|Mystery drones ov...|    0|[mystery, drones,...|[mystery, drones,...|(262144,[6,1934,2...|(262144,[6,1934,2...|[20.6102468017298...|[0.99999999888034...|       0.0|
|Whi